# Train Two-layer NN with MNIST

In [16]:
from torchvision import datasets
from tqdm import tqdm
import numpy as np

In [17]:
def load_mnist(normalize = True, flatten = True, one_hot_label = True):
  mnist_train = datasets.MNIST(root = "./data/", train=True, download=True)
  mnist_test = datasets.MNIST(root = './data/', train=False, download=True)
  print ("mnist_train:\n",mnist_train,"\n")
  print ("mnist_test:\n",mnist_test,"\n")
  print ("Done.")

  mnist_train_img = []
  mnist_train_label = []
  mnist_test_img = []
  mnist_test_label = []

  for train_data in mnist_train:
    train_img = np.array(train_data[0]).astype(np.uint8)
    train_img = train_img.reshape(784)
    mnist_train_img.append(train_img)
    mnist_train_label.append(train_data[1])

  for test_data in mnist_test:
    test_img = np.array(test_data[0]).astype(np.uint8)
    test_img = test_img.reshape(784)
    mnist_test_img.append(test_img)
    mnist_test_label.append(test_data[1])

  mnist_train_img, mnist_train_label = np.array(mnist_train_img), np.array(mnist_train_label)
  mnist_test_img, mnist_test_label = np.array(mnist_test_img), np.array(mnist_test_label)

  if normalize:
    mnist_train_img, mnist_test_img = mnist_train_img.astype(np.float32), mnist_test_img.astype(np.float32)
    mnist_train_img /= 255.0
    mnist_test_img /= 255.0

  if one_hot_label:
    new_train_label = []
    new_test_label = []
    for train_label in mnist_train_label:
      fix_label = np.zeros(10)
      fix_label[train_label] = 1
      new_train_label.append(fix_label)
    
    for test_label in mnist_test_label:
      fix_label = np.zeros(10)
      fix_label[test_label] = 1
      new_test_label.append(fix_label)

    mnist_train_label, mnist_test_label = np.array(new_train_label).astype(np.uint8), np.array(new_test_label).astype(np.uint8)
  
  print("load MNIST done")
  return (mnist_train_img, mnist_train_label), (mnist_test_img, mnist_test_label)
    

In [18]:
def sigmoid(x):
  return 1 / (1+np.exp(-x))

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

def numerical_gradient(f,x):
	h = 1e-4
	grad = np.zeros_like(x)
	for idx in range(x.shape[0]):
		tmp_val = x[idx]

		# f(x+h) 계산
		x[idx] = tmp_val + h
		fxh1 = f(x)

		# f(x-h) 계산
		x[idx] = tmp_val - h
		fxh2 = f(x)

		grad[idx] = (fxh1 - fxh2) / (2*h)
		x[idx] = tmp_val

	return grad
  

In [22]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init=0.01):
    self.parameter = {}
    self.parameter["W1"] = weight_init *  np.random.rand(input_size, hidden_size)
    self.parameter["b1"] = np.zeros(hidden_size)
    self.parameter["W2"] = weight_init * np.random.rand(hidden_size, output_size)
    self.parameter["b2"] = np.zeros(output_size)

  def predict(self,x):
    W1, W2 = self.parameter["W1"], self.parameter["W2"]
    b1, b2 = self.parameter['b1'], self.parameter['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)
    return y

  def loss(self, x, t):
    predict = self.predict(x)
    return cross_entropy(predict,t)

  def accuracy(self, x, t):
    predict = self.predict(x)
    predict = np.argmax(predict, axis=1)
    actual = np.argmax(t, axis=1)

    accuracy = np.sum(predict == actual) / float(x.shape[0])
    return accuracy

  def num_gradient(self,x,t):
    loss_W = lambda W: self.loss(x,t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.parameter["W1"])
    grads['b1'] = numerical_gradient(loss_W, self.parameter['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.parameter['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.parameter['b2'])
    return grads

In [23]:
# 학습구현

def train(batch_size, iter_num, lr=0.1):
  network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)
  train_loss = []
  (train_img, train_label), (test_img, test_label) = load_mnist()
  
  for _ in tqdm(range(iter_num)):
    batch = np.random.choice(train_img.shape[0], batch_size)
    batch_img = train_img[batch]
    batch_label = train_label[batch]

    grad = network.num_gradient(batch_img, batch_label)
    for key in ('W1', 'b1', 'W2', 'b2'):
      network.parameter[key] -= lr * grad[key]

    loss = network.loss(batch_img, batch_label)
    #print(loss)
    train_loss.append(loss)
  
  return train_loss

In [25]:
train(100, 500)

mnist_train:
 Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train 

mnist_test:
 Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data/
    Split: Test 

Done.
load MNIST done


100%|██████████| 500/500 [15:47<00:00,  1.90s/it]


[2.3004869332138167,
 2.3017893252731447,
 2.300749631457003,
 2.301267452202328,
 2.3023043823503753,
 2.303139173575916,
 2.3005966505022313,
 2.303650042392133,
 2.3020809761293215,
 2.3031898792822316,
 2.3024295769640535,
 2.3012083728676433,
 2.3018311223918584,
 2.299641669079567,
 2.300658469629477,
 2.300002260864448,
 2.304928825767728,
 2.3001479797524023,
 2.2994781985602177,
 2.3004408164878583,
 2.301176896657079,
 2.3023686744219294,
 2.303678055825647,
 2.3033065981573135,
 2.3033730577204885,
 2.302881462269628,
 2.300389172504576,
 2.3015728193206515,
 2.300223733796016,
 2.2995117139707473,
 2.2987534297120438,
 2.298864798270532,
 2.296665674817915,
 2.300971358496181,
 2.3005369163498357,
 2.3023266526652373,
 2.2984299623798568,
 2.3021629076438725,
 2.298712075506259,
 2.301109393150455,
 2.2994113437397687,
 2.30209889752612,
 2.301333568068649,
 2.2982256994545938,
 2.3011634987363982,
 2.2972086981625575,
 2.3017088137475556,
 2.3029553740568955,
 2.2986557615